## Sélection des vidéos d'intérêt (pour analyse ultérieure)
#### Sélection sur mots ou séquences clés
#### Sélection sur similarité avec un texte de référence (basée sur comptages simples ou tfidf)

##### Installation librairies et modules tiers

In [1]:
# Installations librairies tiers ici inutiles
!pip install spacy-langdetect
#!pip install spacy-lefff
#!pip install pyldavis
#!pip install swifter

     |████████████████████████████████| 998 kB 4.4 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993430 sha256=8a3ba9fa6bc0e8649b7de19c95150f449c4165e62901eac1d755a80ab9648cf7
  Stored in directory: /root/.cache/pip/wheels/89/79/3b/9885ae7f4308f73c514f96d8574d40d7d8173a27731b674013
Successfully built langdetect


In [2]:
# mais le modèle moyen français de Spacy l'est, car il faut calculer les lemmes du texte de référence
!pip install https://github.com/explosion/spacy-models/releases/download/fr_core_news_md-3.2.0/fr_core_news_md-3.2.0.tar.gz
!pip install fr_core_news_md

     |████████████████████████████████| 46.9 MB 1.3 MB/s 
     |████████████████████████████████| 6.0 MB 5.0 MB/s 
     |████████████████████████████████| 10.1 MB 56.9 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 628 kB 45.9 MB/s 
     |████████████████████████████████| 451 kB 68.1 MB/s 
     |████████████████████████████████| 181 kB 53.7 MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-3.2.0-py3-none-any.whl size=46938366 sha256=0e27898001de2cfb036b59f09527477899a72cd2b1185600d60a692f41c04dd2
  Stored in directory: /root/.cache/pip/wheels/ba/71/98/53c0dd09503b9eefd53afb0b60998dba3b2e41759e0b1b41ce
Successfully built fr-core-news-md
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successf

## Ensuite, REDEMARRER L'ENVIRONNEMENT D'EXECUTION

### Librairies, variables globales...¶

In [1]:
# 1.a
# librairies générales (on ne se servira pas forcément de toutes)
import os
import csv
import re
import math
#import pickle
import datetime as dt
from ast import literal_eval
from tqdm import tqdm

In [2]:
# 1.b
# librairies générales data science
import numpy as np
import pandas as pd
# import swifter
# stats
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau

In [3]:
# 1.c
# pour affichage dtypes pandas quand plus de 60 colonnes
pd.set_option('display.max_rows', 120)
# pour affichage des bars tqdm avec apply et map
tqdm.pandas()

In [4]:
# 1.d
# la librairie graphique la plus simple d'utilisation
import plotly.express as px
import plotly.graph_objects as go

#### librairies text-mining / NLP

In [5]:
# 1.e
import spacy
nlp = spacy.load("fr_core_news_md")

In [6]:
# 1.f
# on ajoute à la pipeline standard de spacy le détecteur de langue
from spacy.language import Language
from spacy_langdetect import LanguageDetector
def get_lang_detector(nlp, name):
    return LanguageDetector()
Language.factory("language_detector", func=get_lang_detector)

<function __main__.get_lang_detector>

In [7]:
# 1.g
nlp.add_pipe('language_detector', last=True)

In [8]:
# 1.h
import gensim
from gensim import corpora

In [9]:
# 1.i imports spécifiques à google colab
from google.colab import files, drive

#### Autres fichiers

In [10]:
# 1.j
SELECTION_MOTS_COURANTS = "sélection_mots_courants.txt"

### Lecture du fichier des 762 vidéos prétraitées
##### Prétraitées : on a effectué au préalable l'analyse nlp de Spacy sur les sous_titres et renseigné :

###### 'langue' langue de la vidéo tel que détecté par spacy (on garde 'fr')
###### 'tkn_sous_titres' listes de token avec un tuple par token, contenant en : 
0 : texte, 1 : index dans doc, 2 : position dans doc, 3 : lemme, 4 : pos, 5 : index dans doc de la tête, 6: la relation de dépendance à la tête, 7 : est un mot stop, 8 : type entité nommée (si existe)
###### 'ent_sous_titres' liste d'entités nommées avec un tuple par entité nommée, contenant en :
0 : texte, 1 : position du début dans doc, 2 : position fin, 3 : type entité nommée

In [11]:
# 2.a passage obligé pour récupérer un fichier sur le drive
# il va falloir choisir le drive (account Google) puis renseigner un mot de passe généré
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# 2.b
chemin_nom_dataframe = "/content/drive/My Drive/Datasets NLP/Climat/videos_climat_nlp.csv"
df_videos = pd.read_csv(chemin_nom_dataframe, sep=',', encoding='utf-8')

In [13]:
# 2.c
# étape de retransformation des chaînes analyse de résultats sauvegardés en liste de tuples
df_videos['tkn_sous_titres'] = df_videos['tkn_sous_titres'].progress_map(lambda x: literal_eval(str(x)))
df_videos['ent_sous_titres'] = df_videos['ent_sous_titres'].progress_map(lambda x: literal_eval(str(x)))

100%|██████████| 762/762 [00:00<00:00, 1987.68it/s]


In [14]:
# 2.d
# nombre de vidéos, liste des colonnes (champs) et leurs types
nb_videos = len(df_videos)
print(nb_videos)
df_videos.dtypes

762


id_video               object
id_chaine              object
titre                  object
description            object
tags                   object
date_chargement        object
durée                 float64
pays                   object
langue                 object
active                float64
ss_titres              object
nom_chaine             object
catégorie_chaine       object
langue_sous_titres     object
tkn_sous_titres        object
ent_sous_titres        object
dtype: object

In [15]:
# 2.e vérification
print(df_videos.iloc[0]['ss_titres'])
print(df_videos.iloc[0]['tkn_sous_titres'])

Une salle d’audition complètement euphorique. Bienvenue au casting de n’oubliez pas les paroles. Ici pas de stresse ou d’inquiétudes, les candidats s’applaudissent silencieusement. Et comme l’émission, ils reprennent les refrains en cœur. « J’étais très surpris. C’est vrai qu’il y a un peu l’ambiance qu’on voit à la télé, mais je ne m’attendais pas ce que ce soit vraiment le cas. Et dès le casting, dès les premières minutes, dès qu’on est rentré dans la salle, en fait tout le monde fait comme s'ils se connaissaient et ça instaure un climat de confiance direct.» Evalués à l’écrit sur leurs connaissances des paroles, les 54 candidats retenus ont ensuite chanté sur une musique connue de leur choix. « On va regarder s’ils sourient, s’ils sont contents d’être là, s’ils sont aussi à l’écoute des autres parce que c’est hyper important. Ils n’ont pas besoin d’avoir des grandes voix, mais de chanter juste et qu’on les imagine sur le plateau avec Nagui. » 20 candidats ont été sélectionnés à l’is

#### Cette structure est assez complète, mais ce dont on a ici besoin, c'est une simple liste des lemmes, dont la sélection se base sur les parties du discours pleines, et on élimine les stop words

##### Fonction qui à partir des données lemme, POS, et stop-word renvoie le lemme ou rien du tout si non retenu

In [16]:
# 2.f
# répartition un peu arbitraire entre catégories vides et pleines
# les premières faisant office de stopwords, dans un modèle sacs de mots
CATEGORIES_VIDES = ['ADP', 'AUX', 'CONJ', 'CCONJ', 'DET', 'PART', 'PRON',
                    'PUNCT', 'SCONJ', 'SYM', 'SPACE']
CATEGORIES_PLEINES = ['ADJ', 'ADV', 'INTJ', 'NOUN', 'NUM', 'PROPN', 'VERB', 'X']

In [17]:
# 2.g
# cette fonction servira à la fois sur les analyses déjà faites et sur le texte de référence utilisé à fins de comparaisons
def selectionne_lemme(lemme, pos, est_stopword):
  selection_lemme = ''
  if pos in CATEGORIES_PLEINES and est_stopword == False:
    selection_lemme = lemme
  return selection_lemme


In [18]:
# 2.h
# utilisation sur les analyses déjà faites, création d'une nouvelle colonne, en utilisant la fonction suivante,
# qui prend en argument la liste des tokens résultat de l'analyse nlp spacy effectuée auparavant
def cree_l_lemmes_sur_resultat_nlp(l_tokens_nlp):
  l_lemmes = []
  for token in l_tokens_nlp:
    lemme = selectionne_lemme(token[3], token[4], token[7])
    if lemme != '':
      l_lemmes.append(lemme)
  return l_lemmes


In [19]:
# 2.i
# appliquer ceci en créant une nouvelle colonne du dataframe stockant la liste des lemmes d'intérêt
df_videos['lemmes_sous_titres'] = df_videos['tkn_sous_titres'].progress_apply(lambda x: cree_l_lemmes_sur_resultat_nlp(x))

100%|██████████| 762/762 [00:00<00:00, 1189.44it/s]


In [20]:
# 2.j vérification
print(df_videos.iloc[0]['lemmes_sous_titres'])

['salle', 'audition', 'complètement', 'euphorique', 'bienvenue', 'casting', 'oublier', 'parole', 'ici', 'stresse', 'inquiétude', 'candidat', 'applaudir', 'silencieusement', 'émission', 'reprendre', 'refrain', 'cœur', 'surprendre', 'vrai', 'ambiance', 'voir', 'télé', 'attendre', 'vraiment', 'cas', 'casting', 'premier', 'minute', 'rentrer', 'salle', 'monde', 'connaître', 'instaure', 'climat', 'confiance', 'direct', 'evalué', 'écrit', 'connaissance', 'parole', '54', 'candidat', 'retenir', 'ensuite', 'chanter', 'musique', 'connaître', 'choix', 'regarder', 'sourire', 'content', 'écoute', 'hyper', 'important', 'besoin', 'grand', 'voix', 'chanter', 'imagine', 'plateau', 'Nagui', '20', 'candidat', 'sélectionner', 'issue', 'journée', 'Commercial', 'étudiant', 'retraité', 'profil', 'multiple', 'point', 'commun', 'chant', 'passion', 'beaucoup', 'émotion', 'beaucoup', 'partage', 'aime', 'beaucoup', 'partager', 'ensemble', 'donner', 'retenir', 'bien', 'visage', 'pouvoir', 'peut-être', 'voir', 'chan

### Recherche sur mots-clés

#### On va d'une part toper (1 ou 0) les vidéos qui présentent la ou les combinaisons de lemmes clés dans les sous titres
On aurait pu aussi considérer les descriptions et les tags, mais dès lors que les combinaisons d'intérêt sont dans les sous-titres, elles le sont aussi dans les descriptions et tags 

#### Et d'autre part mesurer la force des mots clés (la fréquence relative dans les sous-titres)

##### On repère cela à partir de sous-séquences à retrouver dans la liste des lemmes

In [21]:
# 3.a
SOUS_SEQUENCES_CLE = [['réchauffement', 'climatique'], ['gaz', 'effet', 'serre'], ['convention', 'citoyen'], ['émission', 'carbone'],
                     ['transition', 'écologique'], ['manifestation', 'climat'], ['extinction', 'rébellion'], ['CO2'],
                     ['COP21'], ['COP', '21'],['1,5', '°C'], ['1,5°C'], ['transition', 'énergétique']]
                     

##### Fonctions de recherche de sous-séquences

In [22]:
# 3.b
# fonction copiée depuis stackoverflow
# on va chercher une sous-séquence dans celle du texte éclaté en mots
def index_subseq_in_seq(subseq, seq):
    
    i, n, m = -1, len(seq), len(subseq)
    try:
        while True:
            i = seq.index(subseq[0], i + 1, n - m + 1)
            if subseq == seq[i:i + m]:
               return i
    except ValueError:
        return -1

In [23]:
# 3.c
# on peut aussi compter le nombre de séquences
def nb_subseq_in_seq(subseq, seq):
    
    nb_subseq = 0
    remaining_seq = seq
    while len(remaining_seq) >= len(subseq):
        index = index_subseq_in_seq(subseq, remaining_seq)
        if index >= 0:
            remaining_seq = remaining_seq[index + len(subseq):]
            nb_subseq += 1
        else:
            remaining_seq = []
    
    return nb_subseq

##### on créer deux colonnes, une indiquant présence ou non d'une des sous-séquences à détecter, et l'autre fréquence relative de l'ensemble des sous-séquences à rechercher


In [24]:
# 3.d
def presence_sous_sequences(liste_tokens, liste_sous_sequences):
    presence = 0
    for sous_sequence in liste_sous_sequences:
        if index_subseq_in_seq(sous_sequence, liste_tokens) >= 0:
            presence = 1
            break
    return presence  

In [25]:
# 3.e
def lit_presence_termes_cle(l):
    global compteur
    presence_termes_cle = 0
    liste_tokens = l['lemmes_sous_titres']
    if isinstance(liste_tokens, list):
        presence_termes_cle = presence_sous_sequences(liste_tokens, SOUS_SEQUENCES_CLE)
        #print("%05d" % (compteur), end="\r")
        compteur += 1
    return presence_termes_cle

In [26]:
# 3.f
compteur = 0
df_videos['presence_termes_cle'] = df_videos.progress_apply(lambda l: lit_presence_termes_cle(l), axis=1)

100%|██████████| 762/762 [00:00<00:00, 6473.24it/s]


In [27]:
# 3.g
def freq_relative_sous_sequences(liste_tokens, liste_sous_sequences):
    freq_relative = 0.0
    nb_tokens = len(liste_tokens)
    nb_presences = 0
    for sous_sequence in liste_sous_sequences:
        nb_presences += nb_subseq_in_seq(sous_sequence, liste_tokens)
    if nb_tokens > 0:
        freq_relative = 100 * nb_presences / nb_tokens
    return freq_relative

In [28]:
# 3.h
def calcule_freq_relative_termes_cle(l):
    global compteur
    freq_relative_termes_cle = 0
    liste_tokens = l['lemmes_sous_titres']
    if isinstance(liste_tokens, list):
        freq_relative_termes_cle = freq_relative_sous_sequences(liste_tokens, SOUS_SEQUENCES_CLE)
        #print("%05d" % (compteur), end="\r")
        compteur += 1
    return freq_relative_termes_cle

In [29]:
# 3.i
compteur = 0
df_videos['freq_relative_termes_cle'] = df_videos.progress_apply(lambda l: calcule_freq_relative_termes_cle(l), axis=1)

100%|██████████| 762/762 [00:00<00:00, 3635.94it/s]


In [30]:
# 3.j
df_videos_termes_cle = df_videos.loc[df_videos['presence_termes_cle']==1]
print(len(df_videos_termes_cle))

361


In [31]:
# 3.k
df_videos_termes_cle.head()

,id_video,id_chaine,titre,description,tags,date_chargement,durée,pays,langue,active,ss_titres,nom_chaine,catégorie_chaine,langue_sous_titres,tkn_sous_titres,ent_sous_titres,lemmes_sous_titres,presence_termes_cle,freq_relative_termes_cle
1,v6W5ssygrHo,UCO6K_kkdP-lnSCiO3tPx7WA,"F. de Rugy : sur la fiscalité écologique, les ...","François de Rugy, le ministre de la Transition...",NaN,2018-11-09 09:21:36,87.0,fr,fr,1.0,il ya un manque de d'explication dont on peut ...,franceinfo,Chaines ou émissions de médias traditionnels,fr,"[(il, 0, 0, il, PRON, 1, expl:subj, True, 0), ...","[(mme, 459, 462, PER), (cic, 500, 503, ORG), (...","[yer, manque, explication, peut-être, transpar...",1,3.125000
2,ud2uSm5Qs_4,UCDKcvNGkX-1QxNoBt_Ac-zA,"""L'heure est grave, nul ne peut le sous-estime...",Le député insoumis Alexis Corbière a pris la p...,La France insoumise Alexis Corbière gilets jau...,2018-12-03 17:50:42,690.0,fr,fr,1.0,"Mes chers collègues, Madame la Ministre, Messi...",Alexis Corbière,"Discours militants, alternatifs et engagés",fr,"[(Mes, 0, 0, mon, DET, 2, det, True, 0), (cher...","[(Madame la Ministre, 21, 39, PER), (Messieurs...","[cher, collègue, monsieur, ministre, monsieur,...",1,0.220751
3,l_PhbYB42Vs,UCDKcvNGkX-1QxNoBt_Ac-zA,"""Votre loi Fake news est impuissante et ne ser...",Le député Alexis Corbière est intervenu tout à...,Alexis Corbière France insoumise fake news déb...,2018-11-20 19:22:13,321.0,fr,fr,1.0,Mes chers collègues Monsieur le ministre Mesda...,Alexis Corbière,"Discours militants, alternatifs et engagés",fr,"[(Mes, 0, 0, mon, DET, 2, det, True, 0), (cher...","[(Monsieur, 20, 28, PER), (Mesdames, 41, 49, P...","[cher, collègue, Monsieur, ministre, Mesdames,...",1,0.253807
4,YV3KF07SZi8,UC5TpBfjt9l9trdF3jWLg2Gw,Timide début de mobilisation des jeunes pour l...,Plus de 200 jeunes ont manifesté vendredi à Pa...,#Timide #début #mobilisation #jeunes #pour #cl...,2019-02-15 18:21:47,276.0,fr,fr,1.0,Plus de 200 jeunes ont manifesté vendredi à Pa...,Nouvelles,Complotiste confusioniste pamphlétiste,fr,"[(Plus, 0, 0, plus, ADV, 2, advmod, True, 0), ...","[(Paris, 44, 49, LOC), (Belgique, 178, 186, LO...","[200, jeune, manifester, vendredi, Paris, l'""i...",1,1.456311
7,tG-W2jqnpzA,UC5TpBfjt9l9trdF3jWLg2Gw,Inondations en Australie: les sédiments menace...,Les inondations récentes dans le nord de l'Aus...,#Inondations #Australie #sédiments #menacent #...,2019-02-15 10:46:14,258.0,fr,fr,1.0,Les inondations récentes dans le nord de l'Aus...,Nouvelles,Complotiste confusioniste pamphlétiste,fr,"[(Les, 0, 0, le, DET, 1, det, True, 0), (inond...","[(Australie, 43, 52, LOC), (Grande barrière, 1...","[inondation, récent, nord, Australie, provoque...",1,0.555556


##### Aperçu rapide sur quelques vidéos, choisir quelques valeurs de idx dans exécutions de la cellule suivante

In [32]:
# 3.l
idx = 104
l_tokens = df_videos_termes_cle.iloc[idx]['lemmes_sous_titres']
print(l_tokens)
print(len(l_tokens))
print('----------------------')
presence = df_videos_termes_cle.iloc[idx]['presence_termes_cle']
freq_relative = df_videos_termes_cle.iloc[idx]['freq_relative_termes_cle']
print("présence: %d - fréq. relative %.2f" % (presence, freq_relative))

['griveau', 'bonjour', 'bienvenue', 'compte-rendu', 'conseil', 'ministre', 'tenir', 'jour', 'parole', 'suite', 'Nathalie', 'Loiseau', 'occasion', 'faire', 'communication', 'important', 'brexit', 'propose', 'faire', 'évidemment', 'prendre', 'connaissance', 'pouvoir', 'poser', 'question', 'relatif', 'sujet', 'brexit', 'actualité', 'européen', 'charger', 'moment', 'traité', 'resigner', 'hier', 'sujet', 'européen', 'profiter', 'Nathalie', 'Loiseau', 'prendre', 'plaisir', 'répondre', 'question', 'céder', 'parole', 'suite', '-n.', 'Loiseau', 'bonjour', '65', 'jour', 'date', 'choisir', 'Royaume', '-', 'Uni', 'quitter', 'Union', 'européen', '2', 'an', 'mettre', 'oeuvre', 'loyalement', 'décision', 'choix', 'peuple', 'britannique', 'quitter', 'Union', 'européen', 'séparation', 'compliquer', 'séparation', 'réalité', 'scenario', 'séparation', 'ordonner', 'séparation', 'brutal', 'bien', 'changer', 'avis', 'sépare', 'séparation', 'ordonner', 'prévoir', 'accord', 'retrait', 'négocier', 'conclure', 'U

### Pour gensim, utilisé tant pour la similarité que pour l'analyse de topics
#### on va créer la matrice (document, terme) des sous-titres (plus l'équivalent pour le texte de référence)
##### Gensim utilise des dictionnaires associant chaînes de caractères à index en interne, d'où travailler sur les lemmes

#### Mais auparavant, récupération du texte de référence (dépendant du sujet de référence)

In [33]:
# 4.a
chemin_nom_reference = "/content/drive/My Drive/Datasets NLP/Climat/texte référence changement climatique.txt"

In [34]:
# 4.b
fichier_ref = open(chemin_nom_reference, 'r', encoding='utf8')
texte_ref = fichier_ref.read()
fichier_ref.close()

In [35]:
# 4.c vérification
print(texte_ref)

Le réchauffement climatique comprend à la fois le réchauffement induit par les émissions de gaz à effet de serre d'origine humaine et les changements de régimes météorologiques à grande échelle qui en résultent. Bien qu'il y ait eu des périodes précédentes de changement climatique, depuis le milieu du 20e siècle les activités humaines ont eu un impact sans précédent sur le système climatique de la Terre et ont provoqué des changements à l'échelle mondiale. Le principal facteur de réchauffement est l'émission de gaz à effet de serre, dont plus de 90 % sont le dioxyde de carbone (CO2) et le méthane. La combustion de combustibles fossiles comme le charbon, le pétrole et le gaz naturel pour la consommation d'énergie est la principale source de ces émissions, avec des contributions supplémentaires de l'agriculture, de la déforestation et de la production industrielle. La cause humaine du changement climatique n'est contestée par aucun organisme scientifique de renommée nationale ou internat

###### procéder à l'analyse syntaxique du texte de référence

In [36]:
# 4.d
# construction d'une liste de lemmes pour analyses sacs de mots
# l'analyse linguistique a déjà été faite par spacy, on en passe le résultat (tokens_spacy)
def cree_l_lemmes_sur_resultat_brut_spacy(tokens_spacy):

  l_lemmes = []  
  for token in tokens_spacy:
    lemme = selectionne_lemme(token.lemma_, token.pos_, token.is_stop)
    if lemme != '':
      l_lemmes.append(lemme)
  return l_lemmes

In [37]:
# 4.e
# performer l'analyse syntaxique spacy et en déduire le sac de lemmes
tokens_nlp_ref = nlp(texte_ref)
l_tokens_ref_normes = cree_l_lemmes_sur_resultat_brut_spacy(tokens_nlp_ref)
# besoin d'une "anti-normalisation" pour aligner le texte sur les caractéristiques des sous-titres
# s'aligner sur la mauvaise qualité des sous-titres, et passer tout en minuscule
# (mais pas les sigles, à voir)
# et éliminer les symboles '—' et '«' (il n'y a pas la fermante)
l_tokens_ref = []
for token_norme in l_tokens_ref_normes:
    token = token_norme.lower()
    if token not in ['—', '«']:
        l_tokens_ref.append(token)

##### opérations de constructions d'objets à gensim

In [38]:
# 4.f
# parcourir l'ensemble du dataframe pour obtenir les données de la matrice document / termes
# sous la forme d'une liste par documents de vecteurs de termes 
# on stocke une liste de listes de termes (ici les lemmes), vidéo par vidéo
mat_doc_tokens_ss_titres = []

# parcourir les vidéos
for ix_video, l_tokens in tqdm(df_videos['lemmes_sous_titres'].iteritems(), total=df_videos.shape[0]):
    
    mat_doc_tokens_ss_titres.append(l_tokens)
    
# rajouter en dernier le texte de référence
mat_doc_tokens_ss_titres.append(l_tokens_ref)

# création d'un dictionnaire donnant le terme à partir de son identifiant numérique
dico_termes_ss_titres = corpora.Dictionary(mat_doc_tokens_ss_titres)

# conversion en une liste de vecteurs fréquence des termes sur documents 
# représentés sous la forme (id numérique du terme, fréquence du terme dans le document)
# les fréquences à 0 n'y sont pas présentes
mat_doc_termes_freq_ss_titres = [dico_termes_ss_titres.doc2bow(l_tokens) \
                                 for l_tokens in mat_doc_tokens_ss_titres]

100%|██████████| 762/762 [00:00<00:00, 1013656.72it/s]


In [39]:
# 4.g
# examiner les objets créés (et stocker taille de corpus et taille de dico)
taille_corpus = len(mat_doc_tokens_ss_titres)
print("taille du corpus : %d textes" % (taille_corpus))
# vaut le nombre de vidéos dans l'échantillon, plus 1, le texte de référence

# conserver aussi le nombre de vidéos avec présence du terme clé
taille_corpus_avec_termes_cle = len(df_videos_termes_cle)
print("dont vidéos avec termes clé : %d textes" % \
      (taille_corpus_avec_termes_cle))

# prendre deux trois indices (numéros de documents)
idx = 100
print("tokens pour document %d :" % (idx), mat_doc_tokens_ss_titres[idx])
print("termes et fréquences pour document %d :" % (idx), mat_doc_termes_freq_ss_titres[idx])
taille_dico = len(dico_termes_ss_titres)
print("dico termes ss titres : %d termes dans le dictionnaire" % (taille_dico))
for i in range(1000,1010): print(i, dico_termes_ss_titres[i])

taille du corpus : 763 textes
dont vidéos avec termes clé : 361 textes
tokens pour document 100 : ['savoir', 'député', 'vouloir', 'rouler', 'vite', 'Olivier', 'Dassault', 'élu', 'républicain', 'Oise', 'propose', 'aller', '150', 'heure', 'autoroute', 'lieu', '13', 'vite', 'attentif', 'réduire', 'risque', '”', 'ajouter', 'Bourré', 'concentrer', 'peur', 'choper', 'bien', 'droite', 'réfléchir', 'emmerder', 'relax', 'savoir', 'Manuel', 'Valls', 'prendre', 'cher', 'Barcelone', 'ancien', 'ministre', 'vouloir', 'devenir', 'maire', 'capitale', 'catalan', 'originaire', 'population', 'accueillir', 'bras', 'récemment', 'piéger', 'journaliste', 'demander', 'prix', 'ticket', 'métro', 'savoir', 'répondre', 'Manu', 'déconne', 'monde', 'pose', 'question', 'falloir', 'savoir', 'contre', 'dire', 'prix', 'avion', 'low', 'cost', 'falloir', 'envie', 'revenir', 'Déconnez', 'savoir', 'Donald', 'Trump', 'croire', 'changement', 'climatique', '300', 'scientifique', 'rédiger', 'rapport', 'expliquer', 'effort', 'é

In [40]:
print(dico_termes_ss_titres[144])

savoir


#### La simple mesure des fréquences brutes de termes dans les documents, assortis à une bonne mesure comme celle du cosinus qui permet de normaliser la taille des documents, permet déjà d'examiner les documents par rapport à leur similarité avec le texte de référence

#### On va examiner les similarités en étudiant :
1) le comptage simple<p>
2) le tf-idf (en essayant quelques variantes)<p>
3) pour mémoire, le one-hot encoding<p>

#### Mesurer la similarité des documents avec le texte de référence passe par :
- transformer les listes de termes, fréquence (ou poids de façon plus générale) en matrices (creuses)<p>
- calculer le cosinus entre deux documents<p>
- représenter graphiquement ces similarités en les mettant en regard avec les résultats de l'approche par mots clé<p>

#### Utilitaires : 
##### vecteurs creux : transformer une liste de (id, poids) avec les seuls id effectivement présents dans le document, en un np.array de longueur le nombre de termes dans le dictionnaire, pour éviter de calculer en bloc la matrice de similarité
#### Et calculer le cosinus entre deux vecteurs

In [41]:
# 5.a
# A partir d'un vecteur dense de (index, poids) issu de la construction gensim
# poids créer un vecteur sur l'ensemble des composantes du dictionnaire, donc creux (sparse)
def vect_gensim_2_vect_creux_np(vect_poids, taille_dico):
    vect_creux_np = np.zeros(taille_dico)
    for (index, poids) in vect_poids:
        vect_creux_np[index] = poids
    return vect_creux_np

In [42]:
# 5.b
def similarite_cosinus(vect_np_1, vect_np_2):
    cosinus = np.dot(vect_np_1, vect_np_2) / \
              (np.sqrt(np.dot(vect_np_1,vect_np_1)) * np.sqrt(np.dot(vect_np_2,vect_np_2)))
    return cosinus

##### Fonction pour calculer les similarités des sous-titres avec le texte de référence, et ensuite les présenter triées

In [43]:
# 5.c
# pour le calcul des similarités :
# on étend les vecteurs de poids (fréquence, etc.) sur la taille du dictionnaire (10000+ termes)
# et on calcule le cosinus sur le produit scalaire entre le vecteur de référence
# et le vecteur de sous-titres couramment considéré

# pour chaque (sous-titres de) vidéos on stocke un triplet de similarités :
# présence (fréquence absolue) du terme clé, fréquence relative et surtout similarité vectorielle avec le texte de référence
# similarité basée, similarité qui peut être de fréquence absolue, tf-idf, voire one-hot encoding

def calcule_similarites_avec_ref(mat_doc_termes_poids, taille_corpus, taille_dico, df_videos):
    l_similarites = []

    # créer le vecteur sparse pour le texte de référence, qui est en dernier (cas sparse) - en numpy
    vect_creux_ref = vect_gensim_2_vect_creux_np(mat_doc_termes_poids[taille_corpus-1], taille_dico)
                                                 
    # boucler sur les autres vecteurs du corpus et calculer la similarité de cosinus avec le texte de référence
    # et stocker des triplets :
    # (index_video, présence termes clé, fréquence relative termes clé, similarité avec texte de référence)
    # index vidéo pour garder une référence de la vidéo d'origine quand la liste aura été triée
    
    for idx_video, vect_doc_termes_poids in enumerate(mat_doc_termes_poids[:taille_corpus-1]):
        
        # lire les éléments relatifs aux termes clés
        presence_termes_cle = df_videos.iloc[idx_video]['presence_termes_cle']
        freq_relative_termes_cle = df_videos.iloc[idx_video]['freq_relative_termes_cle']
        
        # calculer la similarité cosinus
        vect_creux_compare = vect_gensim_2_vect_creux_np(vect_doc_termes_poids, taille_dico)
        similarite = similarite_cosinus(vect_creux_compare, vect_creux_ref)
                 
        l_similarites.append((idx_video, presence_termes_cle, freq_relative_termes_cle, similarite))

    # trier par similarité, les plus fortes en tête
    l_similarites.sort(key=lambda x: x[3], reverse=True)
    
    return l_similarites

In [44]:
# 5.d
# sur les vidéos triées par ordre de similarité décroissant, noter les "premières" vidéos
# s'il y a N vidéos taggées explicitement des mots clés, les afficher toutes
# et éventuellement compléter de telle sorte qu'il y ait au moins N vidéos en tout
# on va lister des quadruplets (index_video, presence_termes_cle, freq_relative_termes_cle, similarité)

def liste_videos_similaires(l_similarites_triees, nb_videos_presence_explicite):
    l_videos_similaires = []
    idx_video_presence_explicite = 0
    idx_video_presence_non_explicite = 0
    for no_ordre, (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
        in enumerate(l_similarites_triees):
        video_info_similarite = (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite)
        if idx_video_presence_explicite < nb_videos_presence_explicite:
            l_videos_similaires.append(video_info_similarite)
            if presence_termes_cle >= 1:
                idx_video_presence_explicite += 1
            else:
                idx_video_presence_non_explicite += 1
        else:
            if idx_video_presence_non_explicite < nb_videos_presence_explicite:
                l_videos_similaires.append(video_info_similarite)
                idx_video_presence_non_explicite += 1
        if idx_video_presence_explicite >= nb_videos_presence_explicite: 
            break
    return l_videos_similaires

In [45]:
# 5.e
# montre la liste des premières vidéos
def affiche_videos_similaires(l_similarites, nb_videos_presence_explicite):
    l_videos_similaires = liste_videos_similaires(l_similarites, nb_videos_presence_explicite)
    for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) in l_videos_similaires:
        print("vidéo %d\t fréq. relative termes clé : %.2f\t similarité : %.2f" % \
              (idx_video, freq_relative_termes_cle, similarite))
    

In [46]:
# 5.f
# représenter graphiquement la distribution des similarités (sur toutes les videos)

def montre_repartition_similarites(l_similarites_triees, nature_similarite):
    
    # on a besoin de similarité avec présence mot clé explicite ou non
    similarite_explicite = [similarite for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
                            in l_similarites_triees if presence_termes_cle >= 1]
    similarite_non_explicite = [similarite for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
                                in l_similarites_triees if presence_termes_cle < 1]
    # calculer le nombre de bins sur la similarité maximum (celle de la première vidéo)
    # (arrondir au montant sup)
    nb_bins = math.ceil(100 * l_similarites_triees[0][3])
    
    # tracer figure
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=similarite_explicite, nbinsx=nb_bins, name="explicite", marker_color="red"))
    fig.add_trace(go.Histogram(x=similarite_non_explicite, nbinsx=nb_bins, name="non explicite", marker_color="green"))
    fig.update_layout(title=nature_similarite, barmode='stack', xaxis_title="similarité", yaxis_title="nb vidéos")
    fig.show()

In [ ]:
# pour mémoire, la même, mais avec plotly express
def montre_repartition_similarites_px(l_similarites_triees):
    
    # plotly express, il faut passer par un dataframe,
    # on a besoin de similarité et présence mot clé explicite ou non
    similarite = [similarite for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
                  in l_similarites_triees]
    presence_explicite = ["Oui" if presence_termes_cle >= 1 else "Non" \
                for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) in l_similarites_triees]
    dico_similarite = {'similarité': similarite, 'avec termes clé': presence_explicite}
    df_similarite = pd.DataFrame(dico_similarite)
    
    # calculer le nombre de bins sur la similarité maximum (celle de la première vidéo)
    # (arrondir au montant sup)
    nb_bins = math.ceil(100 * df_similarite.iloc[0,0])

    fig = px.histogram(df_similarite, x="similarité", color="avec termes clé",
                       color_discrete_map={'Oui':'red', 'Non':'green'},
                       nbins=nb_bins)
    fig.show()

In [47]:
# 5.g
# représenter graphiquement la relation entre fréquence relative des mots clé dans les sous-titres 
# et similarité avec le texte de référence

def montre_relation_freq_termes_cle_similarite(l_similarites_triees):
    
    # plotly express, il faut passer par un dataframe,
    # on a besoin de similarité et fréquences relatives des termes clé
    # et uniquement sur les vidéos avec présence des termes clé
    similarite = [similarite for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
                  in l_similarites_triees if presence_termes_cle >= 1]
    log_freq_relative = [math.log(freq_relative_termes_cle)\
                for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
                in l_similarites_triees if presence_termes_cle >= 1]
    num_video = [idx_video for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
                 in l_similarites_triees if presence_termes_cle >= 1]
    dict_similarite = {'num. video': num_video,
                       'similarite': similarite,
                       'log fréq. relative': log_freq_relative}
    df_similarite = pd.DataFrame(dict_similarite)
    
    fig = px.scatter(df_similarite, x="log fréq. relative", y="similarite")
    x_text = 0.5 * (max(log_freq_relative) + min(log_freq_relative))
    y_text = 1.25 * max(similarite)
    corr_similarite_freq_relative = pearsonr(log_freq_relative, similarite)
    text_corr = "Correlation %.2f" % (corr_similarite_freq_relative[0])
    fig.update_layout(autosize=False, width=600, height=400,
                      annotations=[go.layout.Annotation(x=x_text, y=y_text, text=text_corr, showarrow=False)])
    fig.show()

#### commencer donc avec les fréquences brutes

In [48]:
# 6.a
l_similarites_freq = calcule_similarites_avec_ref(mat_doc_termes_freq_ss_titres,\
                                                  taille_corpus, taille_dico, df_videos)

In [49]:
# 6.b
# afficher
affiche_videos_similaires(l_similarites_freq, 20)

vidéo 613	 fréq. relative termes clé : 0.14	 similarité : 0.49
vidéo 401	 fréq. relative termes clé : 0.36	 similarité : 0.47
vidéo 571	 fréq. relative termes clé : 0.32	 similarité : 0.44
vidéo 752	 fréq. relative termes clé : 1.00	 similarité : 0.44
vidéo 715	 fréq. relative termes clé : 0.40	 similarité : 0.43
vidéo 115	 fréq. relative termes clé : 2.44	 similarité : 0.43
vidéo 487	 fréq. relative termes clé : 0.86	 similarité : 0.42
vidéo 477	 fréq. relative termes clé : 0.71	 similarité : 0.42
vidéo 499	 fréq. relative termes clé : 0.15	 similarité : 0.40
vidéo 704	 fréq. relative termes clé : 0.99	 similarité : 0.40
vidéo 145	 fréq. relative termes clé : 0.69	 similarité : 0.39
vidéo 296	 fréq. relative termes clé : 0.41	 similarité : 0.38
vidéo 352	 fréq. relative termes clé : 1.07	 similarité : 0.38
vidéo 682	 fréq. relative termes clé : 0.00	 similarité : 0.37
vidéo 421	 fréq. relative termes clé : 0.62	 similarité : 0.36
vidéo 413	 fréq. relative termes clé : 0.00	 similarité

In [50]:
# 6.c
# graphiquement
montre_repartition_similarites(l_similarites_freq, "similarités comptages")

In [53]:
# 6.c
# il peut être intéressant de revenir au sous-titres des vidéos les plus proches du texte de référence
print(df_videos.iloc[40]['ss_titres'])

Je suis cette petite voix qui te trotte dans la tête. Cette angoisse diffuse qui monte quand tu entends parler de réchauffement climatique, de terrorisme et de crise économique. Je suis cette mauvaise conscience qui t’amène parfois à te demander “Et si notre société s’effondrait?”. Pas dans des siècles, mais dans 50 ans, 20 ans ou 5 ans. Je suis cette inquiétude curieuse qui est allée à la rencontre de ceux qui, un peu partout en France, vivent avec la certitude que quelque chose va se passer, et qui s’y préparent. Eux, ce sont les survivalistes. Alors, c’est extrêmement simple. Un tapis de sol, de quoi s’abriter : une bâche, un sac de couchage, des repas tout prêts lyophilisés, un filtre à eau, une gourde et un quart pour pouvoir le mettre sur le feu, une scie pour couper du bois, un petit réchaud à bois, une petite trousse de premiers secours, de bobologie, des allumettes de survie, un peu de ficelle ou de paracorde, une carte de la région, mon passeport. J’ai aussi une boussole, des

In [54]:
# 6.d vérification
print(l_similarites_freq[0])

(613, 1, 0.1443001443001443, 0.4851066237584856)


In [55]:
# 6.e
# ainsi que la corrélation entre similarité et fréquence des termes clés
montre_relation_freq_termes_cle_similarite(l_similarites_freq)

#### Maintenant tf-idf, dans sa formule de base
- fréquence des termes : f (comptage absolu) => notation SMART : n
- fréquence inverse des documents (lissée) : n/N => notation SMART : t
- normalisation longueur des documents : cos => notation SMART : c (remarque, comme on calcule dessus le cosinus, c'est inutile)

In [57]:
# 6.e
# paramétrage des métriques associées à tfidf
param_tfidf = 'ntc'

In [58]:
# 6.f
# on utilise directement la fonction tf-idf de gensim
# il aurait été possible de créer nous même une telle fonction...
#tfidf = gensim.models.TfidfModel(dictionary=dico_termes_ss_titres, smartirs=None)
tfidf = gensim.models.TfidfModel(dictionary=dico_termes_ss_titres, smartirs=param_tfidf)
mat_doc_termes_tfidf_ss_titres = [tfidf[dico_termes_ss_titres.doc2bow(liste_tokens)] \
                                  for liste_tokens in mat_doc_tokens_ss_titres]

In [59]:
# 6.g
l_similarites_tfidf = calcule_similarites_avec_ref(mat_doc_termes_tfidf_ss_titres,\
                                                   taille_corpus, taille_dico, df_videos)

In [60]:
# 6.h
# afficher
affiche_videos_similaires(l_similarites_tfidf, 20)

vidéo 613	 fréq. relative termes clé : 0.14	 similarité : 0.29
vidéo 752	 fréq. relative termes clé : 1.00	 similarité : 0.25
vidéo 477	 fréq. relative termes clé : 0.71	 similarité : 0.25
vidéo 145	 fréq. relative termes clé : 0.69	 similarité : 0.25
vidéo 401	 fréq. relative termes clé : 0.36	 similarité : 0.25
vidéo 715	 fréq. relative termes clé : 0.40	 similarité : 0.24
vidéo 115	 fréq. relative termes clé : 2.44	 similarité : 0.23
vidéo 571	 fréq. relative termes clé : 0.32	 similarité : 0.21
vidéo 232	 fréq. relative termes clé : 0.13	 similarité : 0.20
vidéo 135	 fréq. relative termes clé : 0.00	 similarité : 0.19
vidéo 682	 fréq. relative termes clé : 0.00	 similarité : 0.19
vidéo 616	 fréq. relative termes clé : 1.16	 similarité : 0.19
vidéo 422	 fréq. relative termes clé : 0.91	 similarité : 0.19
vidéo 413	 fréq. relative termes clé : 0.00	 similarité : 0.19
vidéo 718	 fréq. relative termes clé : 0.00	 similarité : 0.18
vidéo 578	 fréq. relative termes clé : 0.00	 similarité

In [61]:
# 6.i
# montrer
montre_repartition_similarites(l_similarites_tfidf, "similarités tf-idf")# 6.f

In [62]:
# 6.j
# ainsi que la corrélation entre similarité et fréquence des termes clés
montre_relation_freq_termes_cle_similarite(l_similarites_tfidf)

##### Essayer avec différents paramètres :
##### Term frequency weighing:
b - binary, t or n - raw, a - augmented, l - logarithm, d - double logarithm, L - log average.
##### Document frequency weighting:
x or n - none, f - idf, t - zero-corrected idf, p - probabilistic idf.
##### Document normalization:
x or n - none, c - cosine, u - pivoted unique, b - pivoted character length.<p>
Par exemple 'afx', 'lfx', 'ltc'.<p>
Présence ou non (one-hot encoding), se traduit par 'bxx' (ou 'bnn')

In [63]:
# 6.k
# mettre cela en fonction
def calcule_similarites_tfidf(dico_termes, mat_doc_tokens, param_tfidf,
                              taille_corpus, taille_dico, df_videos):

    # on utilise directement la fonction tf-idf de gensim
    # il aurait été possible de créer nous même une telle fonction...
    tfidf = gensim.models.TfidfModel(dictionary=dico_termes, smartirs=param_tfidf)
    mat_doc_termes_tfidf = [tfidf[dico_termes.doc2bow(liste_tokens)] \
                                  for liste_tokens in mat_doc_tokens]
    
    # calcul de la similarité
    l_similarites_tfidf = calcule_similarites_avec_ref(mat_doc_termes_tfidf,\
                            taille_corpus, taille_dico, df_videos)
    
    return l_similarites_tfidf

In [ ]:
# 6.l
# on crées une grille à partir de :
# fréquences de termes : n, a, l, L
# fréquence inverse de document : t, p
# on ne regarde que la seule similarité cosinus,
# équivalente (pour nous) à pas de norme puisqu'on calcule ensuite la distance cosinus
liste_tf = ['n', 'a', 'l', 'L']
liste_idf = ['t', 'p']
liste_dn = ['c']

##### Prend un certain temps 

In [ ]:
# 6.m
dico_l_similarites_tfidf = {}

# boucler sur la grille
for tf in liste_tf:
    for idf in liste_idf:
        for dn in liste_dn:
            param_tfidf = tf + idf + dn
            print(param_tfidf)
            
            dico_l_similarites_tfidf[param_tfidf] = \
            calcule_similarites_tfidf(dico_termes_ss_titres, \
                                      mat_doc_tokens_ss_titres, \
                                      param_tfidf, \
                                      taille_corpus, taille_dico, \
                                      df_videos)

ntc
npc
atc
apc
ltc
lpc
Ltc
Lpc


In [ ]:
# 6.n
# fonction renvoyant la liste de toutes les vidéos (tous les index)
# autant ou plus similaires à la vidéo de référence
# qu'une proportion prise sur l'ensemble des vidéos 
# contenant explicitement les termes clés 

# on renvoit une liste de 0 ou de 1 selon les index vidéos
def liste_videos_au_top_explicite(l_similarites_triees, 
                                  proportion_termes_cle_explicite,
                                  taille_corpus,
                                  taille_corpus_avec_termes_cle):
    l_videos_top = [0] * (taille_corpus-1)
    idx_termes_cle_explicite = 0
    for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
        in l_similarites_triees:
        if presence_termes_cle >= 1:
            idx_termes_cle_explicite += 1
        l_videos_top[idx_video] = 1
        if idx_termes_cle_explicite > \
           proportion_termes_cle_explicite * taille_corpus_avec_termes_cle:
            return l_videos_top

In [ ]:
# 6.o
# comme critères d'évaluation pour retenir la meilleure combinaison, on va :
# - considérer la corrélation entre les similarités et 
# les fréquences relatives des termes clés
# - considérer le profil de proportions de vidéos avec mots clés et sans mots clés
# dans les n % vidéos les plus similaires avec la vidéo de référence
proportions_termes_cle_explicite = [0.50, 0.70, 0.80, 0.90]

# boucler sur les paramétrages tf-idf
for tf_idf_dn in dico_l_similarites_tfidf:
    
    print(tf_idf_dn, " :")
    l_similarites_triees = dico_l_similarites_tfidf[tf_idf_dn]
    
    # calcul des corrélations
    similarite = [similarite for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
                  in l_similarites_triees if presence_termes_cle >= 1]
    log_freq_relative = [math.log(freq_relative_termes_cle)\
                for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
                in l_similarites_triees if presence_termes_cle >= 1]
    
    corr_similarite_freq_relative = pearsonr(log_freq_relative, similarite)
    print("corrélation : %.3f" % (corr_similarite_freq_relative[0]))
    
    # calcul des 
    for proportion_termes_cle_explicite in proportions_termes_cle_explicite:
        liste_videos_top = liste_videos_au_top_explicite(l_similarites_triees, 
                                                         proportion_termes_cle_explicite,
                                                         taille_corpus,
                                                         taille_corpus_avec_termes_cle)
        nb_videos_top = sum(liste_videos_top)
        print("pour obtenir %d %% des vidéos explicitement avec termes clé" % \
              (100 * proportion_termes_cle_explicite))
        print("\til faut en tout %2.0f %% des vidéos, dont %2.0f %% avec termes clé" % \
              (100 * nb_videos_top / (taille_corpus-1),\
               100 * proportion_termes_cle_explicite * taille_corpus_avec_termes_cle / nb_videos_top))
    
    print("---------------------------")

ntc  :
corrélation : -0.002
pour obtenir 50 % des vidéos explicitement avec termes clé
	il faut en tout 38 % des vidéos, dont 63 % avec termes clé
pour obtenir 70 % des vidéos explicitement avec termes clé
	il faut en tout 60 % des vidéos, dont 55 % avec termes clé
pour obtenir 80 % des vidéos explicitement avec termes clé
	il faut en tout 73 % des vidéos, dont 52 % avec termes clé
pour obtenir 90 % des vidéos explicitement avec termes clé
	il faut en tout 82 % des vidéos, dont 52 % avec termes clé
---------------------------
npc  :
corrélation : -0.017
pour obtenir 50 % des vidéos explicitement avec termes clé
	il faut en tout 39 % des vidéos, dont 61 % avec termes clé
pour obtenir 70 % des vidéos explicitement avec termes clé
	il faut en tout 59 % des vidéos, dont 56 % avec termes clé
pour obtenir 80 % des vidéos explicitement avec termes clé
	il faut en tout 73 % des vidéos, dont 52 % avec termes clé
pour obtenir 90 % des vidéos explicitement avec termes clé
	il faut en tout 84 % de

#### Manifestement, le meilleur choix est celui des paramètres 'npc'

#### Sélection des vidéos les plus similaires
Nous prenons l'union de :<p>
- des n1 % de l'ensemble des vidéos les mieux classées<p>
- des n2 % de l'ensemble des vidéos explicitement avec termes clé<p>
n1 et n2 se voient d'après les profils affichés plus haut<p>
Par exemple dans l'exemple gilet jaune, on prend les 10 % des vidéos les mieux classées, à compléter, par les vidéos explicitement gilet jaune nécessaires pour arriver à 80 % de couverure


In [ ]:
# 7.a
# fonction renvoyant la liste de toutes les vidéos (tous les index)
# autant ou plus similaires à la vidéo de référence
# qu'une proportion prise sur l'ensemble des vidéos 
# ou bien dans le top pourcentage des vidéos contenant les mots clés

# typiquement proportion_top_similarite est un petit nombre (10 %)
# proportion_termes_cle_explicite un grand nombre (80 %)

# on renvoit une liste de 0 ou de 1 selon les index vidéos
def selectionne_top_videos(l_similarites_triees,
                           proportion_top_similarite,
                           proportion_termes_cle_explicite,
                           taille_corpus,
                           taille_corpus_avec_termes_cle):
    
    l_videos_top = [0] * (taille_corpus-1)
    idx_termes_cle_explicite = 0
    top_similarite_atteint = False
    termes_cle_explicite_atteint = False
    for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
        in l_similarites_triees:
        
        # on est en présence d'une vidéo avec les termes clés
        if presence_termes_cle >= 1:
            idx_termes_cle_explicite += 1
        
        # on a pris les vidéos les plus similaires qu'on voulait
        if idx_video > proportion_top_similarite * (taille_corpus-1):
            top_similarite_atteint = True
        # on a pris les vidéos abec termes clé qu'on voulait
        if idx_termes_cle_explicite > proportion_termes_cle_explicite * taille_corpus_avec_termes_cle:
            termes_cle_explicite_atteint = True
        
        # toper les vidéos qu'on veut conserver
        if presence_termes_cle >= 1 and termes_cle_explicite_atteint == False:
            l_videos_top[idx_video] = 1
        if top_similarite_atteint == False:
            l_videos_top[idx_video] = 1
        
    return l_videos_top
    

#### CHANGER LES VALEURS DE PARAMETRE SELON BESOINS

In [ ]:
# 7.b
# sélection avec les paramètres vus d'après l'exécution précédente
proportion_top_similarite = 0.10
proportion_termes_cle_explicite = 0.80

In [ ]:
# 7.c
l_similarites_triees = dico_l_similarites_tfidf['npc']
liste_videos_top = selectionne_top_videos(l_similarites_triees, 
                                          proportion_top_similarite,
                                          proportion_termes_cle_explicite,
                                          taille_corpus,
                                          taille_corpus_avec_termes_cle)
nb_videos_top = sum(liste_videos_top)
print(nb_videos_top)

284


#### Ecriture du fichier thématique

#### A CHANGER OU NON SELON LE THEME COURANT

In [ ]:
# 7.d
CSV_ECHANTILLON_THEMATIQUE = CSV_ECHANTILLON_THEMATIQUE_GJ

In [ ]:
# 7.e
# à partir des index sélectionnés (mis à 1 dans la liste l_videos_top)
# créer le dataframe des tops vidéos
df_videos['est_top'] = liste_videos_top
df_top_videos = df_videos.loc[df_videos['est_top'] == 1]
print(len(df_top_videos))

560


In [ ]:
# 7.f
df_top_videos.to_csv(CSV_ECHANTILLON_THEMATIQUE, sep=',', encoding='utf-8', header=True,
                     index=False, quoting=csv.QUOTE_ALL)